In [1]:
import sys

# variables
charClass = 0; token = 0; 
lexeme = []; nextChar = ' '; nextToken = ' '; tokenString = ' '
index = 0; chars = []; perLine = []
idNum = 0; constNum = 0; opNum = 0; lineNum = 0
idArray = {}
constValue = 0
add_op = 0; mult_op = 0
grammar = 0; message = ' '
lastIndex = 0

#이번과제에 새로 정의한 global 변수
funcName = []; localvarName = []; varName = []; localLine = []; AR = dict()
num = 0; arIndex = 0
syntax = True; error = False

# Character classes
LETTER = 0; DIGIT = 1; UNKNOWN = 2; EOF = 99

# Token codes
INT_LIT = 10; IDENT = 11; 
LEFT_BRACKET = 20; RIGHT_BRACKET = 21; SEMI_COLON = 22; COMMA = 23; 
VARIABLE = 30; CALL = 31; PRINT_ARI = 32;

#grammar correct
OK = 0; WARNING = 1; ERROR = 2

In [2]:
def getChar():
    # 다음 char를 nextChar에 저장하고 charClass 결정
    global nextChar, charClass, lastRow, lastColumn, index, lastIndex

    if index == lastIndex + 1:
        nextChar = "EOF"
    else:
        nextChar = chars[index]
    index = index + 1
    
    if nextChar == "EOF":
        charClass = EOF
    elif nextChar != ' ' and nextChar != "\n":
        if nextChar.isalpha() or nextChar == '_':
            charClass = LETTER
        elif nextChar.isdigit():
            charClass = DIGIT
        else:
            charClass = UNKNOWN
    else:
        charClass = EOF

In [3]:
def addChar():
    # lexeme 배열에 nextChar를 덧붙임
    global lexeme, nextChar
    lexeme.append(nextChar)

In [4]:
def lexical():
    # 식별자, 숫자, 연산자를 구별해서 nextToken에 셋 중에 어떤 것인지 저장
    global lexeme, charClass, nextToken, idNum, constNum, opNum, lineNum, perLine, tokenString, idArray, constValue, grammar, message
    del lexeme[:]
    getNonBlank()
    
    # 처음 char가 문자로 시작하면 identifier를 의미함, lexeme 문자열을 tokenString에 저장
    if charClass == LETTER:
        addChar(); getChar()
        while charClass == LETTER or charClass == DIGIT:
            addChar(); getChar()
        idNum = idNum + 1
        tokenString = ''.join(lexeme)
        if tokenString == 'variable':
            nextToken = VARIABLE
        elif tokenString == 'call':
            nextToken = CALL
        elif tokenString == 'print_ari':
            nextToken = PRINT_ARI
        else:
            nextToken = IDENT
    
    # 처음 char가 숫자로 시작하면 const를 의미함, lexeme 문자열을 constValue에 저장
#     elif charClass == DIGIT:
#         addChar(); getChar()
#         while charClass == DIGIT:
#             addChar(); getChar()
#         nextToken = INT_LIT
#         constNum = constNum + 1
#         constValue = int(''.join(lexeme))
    
    # 처음 char가 문자, 숫자가 아닌 special character로 시작하면 op를 의미함, lookup 함수 실행
    elif charClass == UNKNOWN:
        lookup(); getChar()
    
    # end of file
    elif charClass == EOF:
        nextToken = EOF
        lexeme.append('EOF0')

In [5]:
def lookup():
    # 문자와 숫자가 아닌 char를 받았을 때 operator(연산자) 구분
    global nextToken, nextChar, opNum, grammar, message
    if nextChar == '{':
        addChar(); nextToken = LEFT_BRACKET
    elif nextChar == '}':
        addChar(); nextToken = RIGHT_BRACKET
    elif nextChar == ',':
        addChar(); nextToken = COMMA
    elif nextChar == ';':
        addChar(); nextToken = SEMI_COLON
    else:
        addChar(); nextToken = EOF

In [6]:
def getNonBlank():
    # white-space 무시
    global nextChar
    while nextChar.isspace():
        getChar()

In [7]:
def start():
    # <start> → <functions>
    functions()
    
def functions():
    # <functions> → <function> { <function> }
    global nextToken, lineNum, perLine, grammar, message
    function()
    while nextToken != EOF:
        function()
        
def function():
    # <function> → <identifier> { <function_body> }
    global nextToken, tokenString, idArray, grammar, message, num, localvarName, localLine, syntax
    localLine.append(num); localLine[num] = list()
    if nextToken == IDENT:
        lexical()
        if tokenString in funcName:
            syntax = False
        else:
            funcName.append(tokenString)
            if nextToken == LEFT_BRACKET:
                lexical()
                function_body()
                if nextToken == RIGHT_BRACKET:
                    lexical()
    varName.append(num)
    varName[num] = localvarName
    num += 1
    localvarName = []
    
def function_body():
    if nextToken == VARIABLE:
        var_definitions()
    statements();
    
def var_definitions():
    while nextToken == VARIABLE:
        var_definition()
    
def var_definition():
    lexical()
    localvarName.append(tokenString)
    lexical()
    while nextToken == COMMA:
        lexical()
        localvarName.append(tokenString)
        lexical()
    if nextToken == SEMI_COLON:
        lexical()
    
#     if nextToken == VARIABLE:
#         lexical()
#         print('variable 다음 변수', tokenString)
#         varName = tokenString
#         print(varName)
#         lexical()
#         while nextToken == COMMA:
#             lexical()
#             varName = tokenString
#             print(varName)
#             lexical()
#         if nextToken == SEMI_COLON:
#             lexical()
#             # 변수명 받기
    
def statements():
    while nextToken != RIGHT_BRACKET:
        statement()
    
def statement():
    # <statement> → call <identifier>; |  print_ari; | <identifier>; 
    global nextToken, tokenString, constValue, expValue, grammar, message, opNum, nextChar, index
    oneLine = tokenString
    if nextToken == CALL:
        lexical()
#         print('call 다음에', tokenString)
        oneLine = oneLine + ' ' + tokenString
        lexical()
#         print('27이여야 정상:', nextToken)
        lexical()
        
    elif nextToken == PRINT_ARI:
        lexical()
        lexical()
    
    elif nextToken == IDENT:
        # 변수명 받기
#         print('statement 내부 변수 부르는 곳, p여야 정상:', tokenString)
        lexical()
#         print('27이여야 정상:', nextToken)
        lexical()
#         print('이제 }(26)이거 출력인가?', nextToken)
    
    localLine[num].append(oneLine)
        

In [8]:
def warningMessage(code):
    if code == 1:
        message = '생략된 등호(=) 삽입'
    elif code == 2:
        message = '중복 연산자(+) 제거'
    elif code == 3:
        message = '중복 연산자(-) 제거'
    elif code == 4:
        message = '중복 연산자(-) 제거'
    elif code == 5:
        message = '중복 연산자(-) 제거'
    elif code == 6:
        message = '생략된 기호(:) 삽입'
    elif code == 7:
        message = '오른쪽 괄호가 없음'
        
    return message

In [9]:
def errorMessage(code):
    if code == 1:
        message = '정의되지 않은 변수가 참조됨'
    elif code == 2:
        message = '식별자(identifier) 뒤에 등호(=)가 없음'
    elif code == 3:
        message = '문장이 식별자로 시작하지 않음'

    return message

In [10]:
def run():
    global funcName, varName, localLine, AR, arIndex
    ARI = list()
#     print('함수들')
#     print(funcName)
    
#     print('변수들')
#     for i in range(num):
#         print(varName[i])
        
#     print('문장들')
#     for i in range(num):
#         print(localLine[i])
    
    main = funcName.index('main')
    
    # AR에 main 함수의 변수 저장
    for i in varName[main]:
        ARI.append('Local variable: ' + str(i))
    AR['main'] = ARI
    
    for i in localLine[main]:
        if i.split()[0] == 'call':
            call('main', i.split()[1], localLine[main].index(i) + 1, arIndex)
        elif i == 'print_ari':
            printARI()
        else:
            varReference(main, i)
            # link count와 local offset 값 출력
            # link count는 링크 따라가는 횟수
            # local offset은 각 AR에서 몇 번째인지

In [11]:
def call(caller, called, returnAddress, dynamicLink):
    global funcName, varName, localLine, AR, arIndex
    
    ARI = list()
    
    if called not in funcName:
        print('Call to undefined function:', called)
        error = False
    
    else:
        funcNum = funcName.index(called)
        # return address 저장 -> 원래 함수 호출문 바로 다음
        ARI.append('Return Address: ' + caller + ':' + str(returnAddress))
        # dynamic link 저장 -> caller의 AR 제일 하단 부분
        ARI.append('Dynamic Link: ' +  str(dynamicLink))
        # local variables 저장
        for i in varName[funcNum]:
            ARI.append('Local variable: ' + str(i))
        
        AR[called] = ARI
        
        if caller != 'main':
            arIndex += 2    
        arIndex += len(varName[funcName.index(caller)])
        
        for i in localLine[funcNum]:
            if i.split()[0] == 'call':
                call(called, i.split()[1], localLine[funcNum].index(i) + 1, arIndex)
            elif i == 'print_ari':
                printARI()
            else:
                varReference(funcNum, i)
    
#     funcNum = funcName.index(called)
#     # return address 저장 -> 원래 함수 호출문 바로 다음
#     ARI.append('Return Address: ' + caller + ':' + str(returnAddress))
#     # dynamic link 저장 -> caller의 AR 제일 하단 부분
#     ARI.append('Dynamic Link: ' +  str(dynamicLink))
#     # local variables 저장
#     for i in varName[funcNum]:
#         ARI.append('Local variable: ' + str(i))
        
#     AR[called] = ARI
        
#     if caller != 'main':
#         arIndex += 2    
#     arIndex += len(varName[funcName.index(caller)])
        
#     for i in localLine[funcNum]:
#         if i.split()[0] == 'call':
#             call(called, i.split()[1], localLine[funcNum].index(i) + 1, arIndex)
#         elif i == 'print_ari':
#             printARI()
#         else:
#             varReference(funcNum, i)
    
#     AR.popitem()

In [12]:
def printARI():
    global funcName, varName, localLine, AR
    
    keys = list(); values = []
    for key in AR.items():
        keys.append(key)
    
    for key, value in keys[::-1]:
        print(key, end = '')
        values = AR[key][::-1]
        for value in values:
            print('\t', value)
    
    print()

In [13]:
def varReference(funcNum, var):
    global AR, funcName, varName
    keys = list()
    # funcNum는 함수의 정의된 순서 번호, first, second, main 순서 -> second이므로 1
    # varName은 funcName과 인덱스가 같음
    # var은 변수명
    
    linkCount = 0
    funcNow = funcName[funcNum]
    
    for key, value in AR.items():
        keys.append(key)
    # keys에는 AR에 담겨있는 순서대로 함수명이 있다
    # 처음 main, first, second
    arNum = keys.index(funcNow) # -> second는 keys에서 3번째이므로 arNum은 2
    
    while True:
        if var in varName[funcNum]: # varName[1], 즉 second의 변수 리스트에 var이 있느냐
            break
        else:
            arNum -= 1
            funcNum = funcName.index(keys[arNum])
            # arNum에서 -1을 해서 AR 순서에서 하나 내려가서 first가 됨 즉, first의 정의된 인덱스를 funcNum에 저장
            linkCount += 1
            
    offset = varName[funcNum].index(var)
    
    if arNum != 0:
        offset += 2
    
    print(funcNow + ':', var, '=>', str(linkCount)+',', offset, '\n')
        

In [14]:
def has_duplicates(seq):
    return len(seq) != len(set(seq))

In [15]:
try:
#     with open(sys.argv[1], "r") as file:
    with open('txt1.txt', "r") as file:
        list_line = file.read()
        for line in range(0, len(list_line)):
            chars.append(list_line[line])
    
    lastIndex = len(list_line) - 1
    lineNum = len(list_line.split('\n'))
    
    for per in range(0, lineNum):
        perLine.append(list_line.split('\n')[per])
    lineNum = 0

    getChar()
    lexical()
    start()
    
    if syntax == True:
        print('Syntax O.K.\n')
    else:
        print('Syntax Error')
        
    if 'main' not in funcName:
        error = True
        print('No Starting function')
        
    if has_duplicates(funcName):
        print('Duplicate declaration of the function name:')
        
    if error == False:
        run()
    
        
except Exception as e:
    print('ERROR - ', e)
    sys.exit()

Syntax O.K.

second: b => 1, 3 

second	 Local variable: a
	 Local variable: x
	 Dynamic Link: 2
	 Return Address: first:1
first	 Local variable: c
	 Local variable: b
	 Local variable: a
	 Dynamic Link: 0
	 Return Address: main:1
main	 Local variable: q
	 Local variable: p

second: q => 2, 1 

first: p => 1, 0 



In [16]:
# keys = []
# for key, value in AR.items():
#     keys.append(key)

In [17]:
# funcName=[]
# num = 0
# funcName.append('first')

# ar=[]

# ar.append(num)
# ar[num] = list()
# ar[num].append('Local variable: p')
# ar[num].append('Local variable: q')

# funcName.append('second')
# num += 1
# ar.append(num)
# ar[num] = list()
# ar[num].append('Return Address: main:1')
# ar[num].append('Dynamic Link: 1')

# ar[1]

In [18]:
# a = [1, 2, 3]
# c = [10, 11, 12]
# b = []
# num = 0
# b.append('first')
# b[num] = a
# b[num]
# num += 1
# b.append('sec')
# b[num] = c
# b[num]
# b[1]

In [19]:
# dic = dict(); arr = list(); arr2 = list()
# dic['a'] = 1
# arr.append(1)
# arr.append(2)
# arr.append(3)
# dic['a'] = arr

# arr2.append(7)
# arr2.append(8)
# arr2.append(9)
# dic['b'] = arr2

# dic.popitem()[0]